Beam Search with Process Reward Model for LLM Reasoning
===

**Dr Chao Shu (chao.shu@qmul.ac.uk)**


This notebook demonstrates a simplified implementation of beam search with Process Reward Models for improving LLM reasoning, based on the algorithm proposed in [Snell et al., 2024](https://arxiv.org/pdf/2408.03314). Beam search explores multiple reasoning paths simultaneously and uses a Process Reward Model (PRM) to evaluate the quality of each step.

Key characteristics in the algorithm:

1. **Multiple Paths**: Beam search explores multiple reasoning paths simultaneously.
2. **Step-by-Step Scoring**: A Process Reward Model scores each step to guide the search.
3. **Path Pruning**: Low-scoring paths are pruned to focus computational resources.
4. **Better Reasoning**: By exploring multiple paths, we can find better solutions than single-shot generation.

In this implementation, we'll use two models:
- **Search Model**: Llama-3.2-1B-Instruct (can be replaced with larger models like 3B or 7B for better results)
- **Process Reward Model**: RLHFlow/Llama3.1-8B-PRM-Deepseek-Data (trained for mathematical reasoning evaluation)

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass, field
from typing import List, Optional, Dict, Tuple, Any
from transformers import AutoModelForCausalLM, AutoTokenizer
from vllm import LLM, SamplingParams
from tqdm.notebook import tqdm
import copy

## 1. Define Core Data Structures

In [2]:
@dataclass
class Beam:
    """Represents a single beam (reasoning path) in the search process."""
    prompt: str
    index: int
    current_text: str = ""
    next_texts: Optional[List[str]] = None
    stop_reasons: Optional[List[str]] = None  # Added stop_reasons field
    pruned: bool = False
    completed: bool = False
    history: List[str] = field(default_factory=list)
    scores: List[float] = field(default_factory=list)

## 2. Load Models with vLLM and Transformers

In [3]:
# Load the LLM using vLLM for efficient GPU inference
def load_llm():
    model_name = "meta-llama/Llama-3.2-1B-Instruct"
    # model_name = "Qwen/Qwen2.5-1.5B-Instruct"
    
    print(f"Loading LLM with vLLM: {model_name}...")
    model = LLM(
        model=model_name,
        tensor_parallel_size=1,  # Number of GPUs to use for tensor parallelism
        gpu_memory_utilization=0.7,  # GPU memory usage limit (leaving some for PRM)
        enable_prefix_caching=True,  # Improves efficiency for beam search
        seed=42
    )
    
    return model

# Load the Process Reward Model (PRM) from the search-and-learn project
def load_prm():
    # Using the RLHFlow PRM model
    model_name = "RLHFlow/Llama3.1-8B-PRM-Deepseek-Data"
    
    print(f"Loading PRM: {model_name}...")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
        device_map="auto"
    )
    
    # Set up for dialogue-based scoring
    tokenizer.padding_side = "right"
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id
    
    # Get the token IDs for + and - (used in scoring)
    plus_tag_id = tokenizer.encode("+")[-1]
    minus_tag_id = tokenizer.encode("-")[-1]
    candidate_tokens = [plus_tag_id, minus_tag_id]
    
    return model, tokenizer, candidate_tokens

In [4]:
# Load the models
llm = load_llm()
prm, prm_tokenizer, candidate_tokens = load_prm()

Loading LLM with vLLM: meta-llama/Llama-3.2-1B-Instruct...
INFO 04-01 06:11:03 __init__.py:207] Automatically detected platform cuda.
INFO 04-01 06:11:15 config.py:549] This model supports multiple tasks: {'reward', 'score', 'classify', 'generate', 'embed'}. Defaulting to 'generate'.
WARNING 04-01 06:11:15 arg_utils.py:1187] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 04-01 06:11:15 config.py:1555] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 04-01 06:11:15 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='meta-llama/Llama-3.2-1B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-1B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-01 06:11:32 model_runner.py:1115] Loading model weights took 2.3185 GB
INFO 04-01 06:11:33 worker.py:267] Memory profiling takes 0.99 seconds
INFO 04-01 06:11:33 worker.py:267] the current vLLM instance can use total_gpu_memory (44.55GiB) x gpu_memory_utilization (0.70) = 31.18GiB
INFO 04-01 06:11:33 worker.py:267] model weights take 2.32GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.18GiB; the rest of the memory reserved for KV Cache is 27.63GiB.
INFO 04-01 06:11:33 executor_base.py:111] # cuda blocks: 56582, # CPU blocks: 8192
INFO 04-01 06:11:33 executor_base.py:116] Maximum concurrency for 131072 tokens per request: 6.91x
INFO 04-01 06:11:36 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_u

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:18<00:00,  1.92it/s]

INFO 04-01 06:11:55 model_runner.py:1562] Graph capturing finished in 18 secs, took 0.15 GiB
INFO 04-01 06:11:55 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 23.01 seconds


Loading PRM: RLHFlow/Llama3.1-8B-PRM-Deepseek-Data...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


## 3. Core Functions for Generation and Scoring

In [5]:
def create_chat_prompt(prompt, current_text="", system_prompt="Solve problems step by step with thorough reasoning.", 
                       custom_chat_template=None, add_generation_prompt=True, continue_final_message=False):
    """Create a prompt formatted for chat models."""
    conversation = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    
    # Add current text as assistant response if it exists
    if current_text:
        conversation.append({"role": "assistant", "content": current_text})

    print("Gen conversation: \n", conversation)
    # print(f"add or continue: {add_generation_prompt}, {continue_final_message}")
    
    # Apply chat template
    tokenizer = llm.get_tokenizer()
    
    # Apply custom chat template if provided
    if custom_chat_template is not None:
        # Save original template
        original_template = tokenizer.chat_template 
        # Set custom template
        tokenizer.chat_template = custom_chat_template
    
    # Apply the template
    input_text = tokenizer.apply_chat_template(
        conversation, 
        tokenize=False,
        add_generation_prompt=add_generation_prompt,
        continue_final_message=continue_final_message
    )
    
    # Restore original template if we changed it
    if custom_chat_template is not None:
        tokenizer.chat_template = original_template
    
    return input_text

def generate_step(prompt, current_text, model, temperature=0.7, custom_chat_template=None, 
                  iter=0):
    """Generate the next reasoning step using vLLM."""
    # Use the system prompt from Meta
    system_prompt = "Solve the following math problem efficiently and clearly:\n\n- For simple problems (2 steps or fewer):\nProvide a concise solution with minimal explanation.\n\n- For complex problems (3 steps or more):\nUse this step-by-step format:\n\n## Step 1: [Concise description]\n[Brief explanation and calculations]\n\n## Step 2: [Concise description]\n[Brief explanation and calculations]\n\n...\n\nRegardless of the approach, always conclude with:\n\nTherefore, the final answer is: $\\boxed{answer}$. I hope it is correct.\n\nWhere [answer] is just the final number or expression that solves the problem."
    
    if iter == 0:
        add_generation_prompt = True
        continue_final_message = False
    else:
        add_generation_prompt = False
        continue_final_message = True

    input_text = create_chat_prompt(prompt, current_text, system_prompt, custom_chat_template, 
                                    add_generation_prompt, continue_final_message)
    
    # Configure sampling parameters
    if iter == -1:  # Last iteration, generate to EOS
        sampling_params = SamplingParams(
            temperature=temperature,
            max_tokens=2048,  # Short steps
            top_p=0.95,
        )
    else:  # Intermediate steps, generate until double newlines
        sampling_params = SamplingParams(
            temperature=temperature,
            max_tokens=2048,  # Short steps
            top_p=0.95,
            stop=["\n\n"], # Step delimiter
            include_stop_str_in_output=True,  # Keep the delimiter in output
        )
    
    # Generate with vLLM
    outputs = model.generate(input_text, sampling_params)
    
    # Get generated text and stop reason
    generated_text = outputs[0].outputs[0].text
    stop_reason = outputs[0].outputs[0].stop_reason  # Corrected from finish_reason to stop_reason
    
    return generated_text, stop_reason

def score_step(prompt, text, model, tokenizer, candidate_tokens):
    """Score the quality of a reasoning path by evaluating each step individually.
    This implementation follows the approach used in the search-and-learn project,
    creating a proper conversation format for the PRM to evaluate each step."""

    print(f"\n\n score_step: receive text: {text}")
    
    # Split the reasoning path into steps at double newlines
    steps = text.split("\n\n")
    
    # Remove any empty steps resulting from the split
    steps = [s for s in steps if s.strip()]

    # print("score_step: steps found: ", steps)
    
    if not steps:
        return [0.5]  # Return neutral score if no steps found
        print("score_step: empty step")
    
    # Evaluate each step in the reasoning path
    step_scores = []
    conversation = []
    
    for k in range(len(steps)):
        if k == 0:
            # For the first step, combine the question with the first reasoning step
            # This provides full context to evaluate the reasoning start
            step_text = prompt + " " + steps[0]
        else:
            # For subsequent steps, we only need the new step text
            step_text = steps[k]
        
        # Build conversation for this step
        conversation.append({"content": step_text, "role": "user"})
        conversation.append({"content": "+", "role": "assistant"})

        print("score_step: conversation: ", conversation)
        
        # Apply chat template
        input_ids = tokenizer.apply_chat_template(
            conversation, return_tensors="pt"
        ).to(model.device)
        
        # Get scores
        with torch.no_grad():
            logits = model(input_ids).logits[:, -3, candidate_tokens]
            scores = logits.softmax(dim=-1)[:, 0]  # Probability of + token
            print("score_step: step score ", scores[0])
            step_scores.append(
                scores[0].detach().to("cpu", dtype=torch.float32).item()
            )

    print("score_step: final score list", step_scores)
    return step_scores

## 4. Beam Search Implementation

In [6]:
def beam_search(problem, config):
    """Perform beam search to solve a problem."""
    # Extract config parameters
    llm_model = config["llm_model"]
    prm_model = config["prm_model"]
    prm_tokenizer = config["prm_tokenizer"]
    candidate_tokens = config["candidate_tokens"]
    total_beams = config["total_beams"]  # Total number of beams (N)
    beam_width = config.get("beam_width", 2)  # Number of expansions per beam (M), default to 2
    num_iterations = config["num_iterations"]  # Number of reasoning steps
    temperature = config["temperature"]
    custom_chat_template = config.get("custom_chat_template", None)
    
    # Print configuration
    print(f"==== Beam Search Configuration ====")
    print(f"Total beams (N): {total_beams}")
    print(f"Beam width (M): {beam_width}")
    print(f"Number of iterations: {num_iterations}")
    print(f"Temperature: {temperature}")
    print(f"Problem: {problem}")
    print("==================================\n")
    
    # Initialize beams
    beams = []
    for i in range(total_beams):
        beams.append(Beam(
            prompt=problem,
            index=i,
            current_text="",
            pruned=False,
            completed=False,
            stop_reasons=None,
            next_texts=None
        ))
    
    completed_beams = []
    
    # Main beam search loop
    for i in tqdm(range(num_iterations), desc="Beam search iterations"):
        print(f"\n\n===== Iteration {i+1}/{num_iterations} =====")
        
        # Get active beams (not pruned or completed)
        active_beams = [b for b in beams if not b.pruned and not b.completed]
        print(f"Active beams at start: {len(active_beams)}")
        
        if not active_beams: 
            print("No active beams remaining. Exiting search.")
            break
        
        # Duplicate active beams to ensure we have total_beams beams if needed
        if len(active_beams) < total_beams:
            repeats = (total_beams // len(active_beams)) + 1
            print(f"Extending active_beams with {repeats} repetitions to reach size {total_beams}")
            active_beams = [copy.deepcopy(b) for b in (active_beams * repeats)[:total_beams]]
            print(f"Active beams after extension: {len(active_beams)}")
        
        # Generate one step for each beam
        print("\n-- Generating next steps for each beam --")
        if i == (num_iterations - 1):
            iter = -1  # Indicate the last iteration
        else:
            iter = i
        for beam_idx, beam in enumerate(active_beams):
            print(f"\nBeam {beam_idx+1}/{len(active_beams)} - Current length: {len(beam.current_text)} chars")
            next_text, stop_reason = generate_step(
                beam.prompt,
                beam.current_text,
                llm_model,
                temperature,
                custom_chat_template,
                iter=iter,
            )
            
            # stop_reason: The stop string or token id that caused the completion
            # to stop, None if the completion finished for some other reason
            # including encountering the EOS token.
            if stop_reason == "\n\n":
                reason_description = "step_complete"  # Hit our paragraph delimiter
            # elif stop_reason is None and len(generated_text) >= sampling_params.max_tokens:
                # reason_description = "max_length"
            elif stop_reason is None:
                reason_description = "eos_token"  # Model decided to stop
            else:
                reason_description = "unknown"
            
            beam.next_texts = [next_text]
            beam.stop_reasons = [reason_description]
            beam.current_text += next_text
            beam.history.append(next_text)

            print(f"  Generated: {next_text[:50]}..." if len(next_text) > 50 else f"  Generated: {next_text}")
            print(f"  Stop reason: {reason_description}")
            
            # Check for completion using stop reasons
            if (
                reason_description == "eos_token" or  # vLLM uses "stop" instead of "EOS"
                # stop_reason == "length" or
                next_text == ""
            ):
                beam.completed = True
                completed_beams.append(copy.deepcopy(beam))
                print(f"  Beam completed with stop reason: {reason_description}")
                    
        # Count completed beams in this iteration
        newly_completed = len([b for b in active_beams if b.completed])
        print(f"\nBeams completed in this iteration: {newly_completed}")
        print(f"Total completed beams so far: {len(completed_beams)}")
        
        # Score each beam
        print("\n-- Scoring beams --")
        prompts, completions = [], []
        non_completed_beams = [b for b in active_beams if not b.completed]
        print(f"Scoring {len(non_completed_beams)} non-completed beams")
        
        for beam in non_completed_beams:
            prompts.append(beam.prompt)
            completions.append([beam.current_text])
        
        # Score each active, non-completed beam
        for i, beam in enumerate(non_completed_beams):
            scores = score_step(
                prompts[i], 
                completions[i][0], 
                prm_model, 
                prm_tokenizer,
                candidate_tokens
            )
            beam.scores = scores
            print(f"  Beam {i+1} scores: {scores}")
        
        # Filter active beams (not completed)
        active_beams = non_completed_beams
        
        if not active_beams:
            print("All beams completed. Exiting search.")
            break
        
        # Calculate how many beams to keep (N/M ratio)
        # For the next iteration, we want to keep top_k beams
        top_k = max(1, total_beams // beam_width)
        print(f"\n-- Pruning beams: keeping top {top_k} of {len(active_beams)} --")
        
        # Sort by score (using the last/most recent step score)
        active_beams = sorted(
            active_beams,
            key=lambda b: b.scores[-1] if b.scores else 0,
            reverse=True
        )
        
        # Print top beam scores
        for i, beam in enumerate(active_beams[:min(3, len(active_beams))]):
            score = beam.scores[-1] if beam.scores else 0
            print(f"  Top beam {i+1}: score = {score:.4f}")
        
        # Keep only the top_k beams
        top_beams = active_beams[:top_k]
        
        # Mark remaining beams as pruned
        for beam in active_beams[top_k:]:
            beam.pruned = True
        
        print(f"  Keeping {len(top_beams)} beams, pruned {len(active_beams) - len(top_beams)} beams")
        
        # Update our beams for next iteration
        beams = top_beams
    
    # Combine completed and remaining beams
    all_beams = completed_beams + [b for b in beams if not b.pruned]
    print(f"\n===== Search Complete =====")
    print(f"Total beams: {len(all_beams)} (Completed: {len(completed_beams)}, Active: {len([b for b in beams if not b.pruned])})")
    
    # If we have more than total_beams, keep only the best ones
    if len(all_beams) > total_beams:
        print(f"Too many beams ({len(all_beams)}), keeping top {total_beams}")
        all_beams = sorted(
            all_beams,
            key=lambda b: b.scores[-1] if b.scores else 0,
            reverse=True
        )[:total_beams]
    # If we have less than total_beams, duplicate some
    elif len(all_beams) < total_beams and len(all_beams) > 0:
        repeats = (total_beams // len(all_beams)) + 1
        print(f"Too few beams ({len(all_beams)}), duplicating to reach {total_beams}")
        all_beams = [copy.deepcopy(b) for b in (all_beams * repeats)[:total_beams]]
    
    # Sort by final score
    all_beams = sorted(
        all_beams,
        key=lambda b: b.scores[-1] if b.scores else 0,
        reverse=True
    )
    
    # Print final results
    print("\n----- Final Results -----")
    for i, beam in enumerate(all_beams[:min(3, len(all_beams))]):
        score = beam.scores[-1] if beam.scores else 0
        print(f"Beam {i+1}: last_score = {score:.4f}, completed = {beam.completed}, steps = {len(beam.history)}")
        # print(f"First 100 chars: {beam.current_text[:100]}...")
        print(f"Full response: {beam.current_text}\n\n")
    print("------------------------")
    
    return all_beams

## 5. Run Beam Search on a Math Problem

In [7]:
# Set up configuration
config = {
    "llm_model": llm,
    "prm_model": prm,
    "prm_tokenizer": prm_tokenizer,
    "candidate_tokens": candidate_tokens,
    "total_beams": 4,  # Total number of beams to maintain
    "beam_width": 2,  # Number of beams to maintain
    "num_iterations": 5,  # Number of reasoning steps
    "temperature": 0.8,  # Temperature for generation
    "custom_chat_template": '{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- if strftime_now is defined %}\n        {%- set date_string = strftime_now("%d %b %Y") %}\n    {%- else %}\n        {%- set date_string = "26 Jul 2024" %}\n    {%- endif %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0][\'role\'] == \'system\' %}\n    {%- set system_message = messages[0][\'content\']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = "" %}\n{%- endif %}\n\n{#- System message #}\n{{- "<|start_header_id|>system<|end_header_id|>\\n\\n" }}\n{%- if tools is not none %}\n    {{- "Environment: ipython\\n" }}\n{%- endif %}\n{{- "Cutting Knowledge Date: December 2023\\n" }}\n{{- "Today Date: " + date_string + "\\n\\n" }}\n{%- if tools is not none and not tools_in_user_message %}\n    {{- "You have access to the following functions. To call a function, please respond with JSON for a function call." }}\n    {{- \'Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.\' }}\n    {{- "Do not use variables.\\n\\n" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- "\\n\\n" }}\n    {%- endfor %}\n{%- endif %}\n{{- system_message }}\n{{- "<|eot_id|>" }}\n\n{#- Custom tools are passed in a user message with some extra guidance #}\n{%- if tools_in_user_message and not tools is none %}\n    {#- Extract the first user message so we can plug it in here #}\n    {%- if messages | length != 0 %}\n        {%- set first_user_message = messages[0][\'content\']|trim %}\n        {%- set messages = messages[1:] %}\n    {%- else %}\n        {{- raise_exception("Cannot put tools in the first user message when there\'s no first user message!") }}\n{%- endif %}\n    {{- \'<|start_header_id|>user<|end_header_id|>\\n\\n\' -}}\n    {{- "Given the following functions, please respond with a JSON for a function call " }}\n    {{- "with its proper arguments that best answers the given prompt.\\n\\n" }}\n    {{- \'Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.\' }}\n    {{- "Do not use variables.\\n\\n" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- "\\n\\n" }}\n    {%- endfor %}\n    {{- first_user_message + "<|eot_id|>"}}\n{%- endif %}\n\n{%- for message in messages %}\n    {%- if not (message.role == \'ipython\' or message.role == \'tool\' or \'tool_calls\' in message) %}\n        {{- \'<|start_header_id|>\' + message[\'role\'] + \'<|end_header_id|>\\n\\n\'+ message[\'content\'] + \'<|eot_id|>\' }}\n    {%- elif \'tool_calls\' in message %}\n        {%- if not message.tool_calls|length == 1 %}\n            {{- raise_exception("This model only supports single tool-calls at once!") }}\n        {%- endif %}\n        {%- set tool_call = message.tool_calls[0].function %}\n        {{- \'<|start_header_id|>assistant<|end_header_id|>\\n\\n\' -}}\n        {{- \'{"name": "\' + tool_call.name + \'", \' }}\n        {{- \'"parameters": \' }}\n        {{- tool_call.arguments | tojson }}\n        {{- "}" }}\n        {{- "<|eot_id|>" }}\n    {%- elif message.role == "tool" or message.role == "ipython" %}\n        {{- "<|start_header_id|>ipython<|end_header_id|>\\n\\n" }}\n        {%- if message.content is mapping or message.content is iterable %}\n            {{- message.content | tojson }}\n        {%- else %}\n            {{- message.content }}\n        {%- endif %}\n        {{- "<|eot_id|>" }}\n    {%- endif %}\n{%- endfor %}\n{%- if add_generation_prompt %}\n    {{- \'<|start_header_id|>assistant<|end_header_id|>\\n\\n\' }}\n{%- endif %}\n'
    # "custom_chat_template": None
}

# Define a math problem
problem = "What is the smallest positive perfect cube that can be written as the sum of three consecutive integers?"

# Run beam search
results = beam_search(problem, config)

==== Beam Search Configuration ====
Total beams (N): 4
Beam width (M): 2
Number of iterations: 5
Temperature: 0.8
Problem: What is the smallest positive perfect cube that can be written as the sum of three consecutive integers?



Beam search iterations:   0%|          | 0/5 [00:00<?, ?it/s]



===== Iteration 1/5 =====
Active beams at start: 4

-- Generating next steps for each beam --

Beam 1/4 - Current length: 0 chars
Gen conversation: 
 [{'role': 'system', 'content': 'Solve the following math problem efficiently and clearly:\n\n- For simple problems (2 steps or fewer):\nProvide a concise solution with minimal explanation.\n\n- For complex problems (3 steps or more):\nUse this step-by-step format:\n\n## Step 1: [Concise description]\n[Brief explanation and calculations]\n\n## Step 2: [Concise description]\n[Brief explanation and calculations]\n\n...\n\nRegardless of the approach, always conclude with:\n\nTherefore, the final answer is: $\\boxed{answer}$. I hope it is correct.\n\nWhere [answer] is just the final number or expression that solves the problem.'}, {'role': 'user', 'content': 'What is the smallest positive perfect cube that can be written as the sum of three consecutive integers?'}]



Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.01it/s, est. speed input: 360.69 toks/s, output: 91.18 toks/s]


  Generated: ## Step 1: Understand the requirements
We need to ...
  Stop reason: step_complete

Beam 2/4 - Current length: 0 chars
Gen conversation: 
 [{'role': 'system', 'content': 'Solve the following math problem efficiently and clearly:\n\n- For simple problems (2 steps or fewer):\nProvide a concise solution with minimal explanation.\n\n- For complex problems (3 steps or more):\nUse this step-by-step format:\n\n## Step 1: [Concise description]\n[Brief explanation and calculations]\n\n## Step 2: [Concise description]\n[Brief explanation and calculations]\n\n...\n\nRegardless of the approach, always conclude with:\n\nTherefore, the final answer is: $\\boxed{answer}$. I hope it is correct.\n\nWhere [answer] is just the final number or expression that solves the problem.'}, {'role': 'user', 'content': 'What is the smallest positive perfect cube that can be written as the sum of three consecutive integers?'}]


...

  Generated: ## Step 3: Since we are looking for the smallest p...
  Stop reason: eos_token
  Beam completed with stop reason: eos_token

Beam 3/4 - Current length: 532 chars
Gen conversation: 
 [{'role': 'system', 'content': 'Solve the following math problem efficiently and clearly:\n\n- For simple problems (2 steps or fewer):\nProvide a concise solution with minimal explanation.\n\n- For complex problems (3 steps or more):\nUse this step-by-step format:\n\n## Step 1: [Concise description]\n[Brief explanation and calculations]\n\n## Step 2: [Concise description]\n[Brief explanation and calculations]\n\n...\n\nRegardless of the approach, always conclude with:\n\nTherefore, the final answer is: $\\boxed{answer}$. I hope it is correct.\n\nWhere [answer] is just the final number or expression that solves the problem.'}, {'role': 'user', 'content': 'What is the smallest positive perfect cube that can be written as the sum of three consecutive integers?'}, {'role': 'assistant', 'content': "


Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.58s/it, est. speed input: 84.96 toks/s, output: 161.54 toks/s]


  Generated: ## Step 3: Express the sum as a perfect cube.

We ...
  Stop reason: eos_token
  Beam completed with stop reason: eos_token

Beam 4/4 - Current length: 502 chars
Gen conversation: 
 [{'role': 'system', 'content': 'Solve the following math problem efficiently and clearly:\n\n- For simple problems (2 steps or fewer):\nProvide a concise solution with minimal explanation.\n\n- For complex problems (3 steps or more):\nUse this step-by-step format:\n\n## Step 1: [Concise description]\n[Brief explanation and calculations]\n\n## Step 2: [Concise description]\n[Brief explanation and calculations]\n\n...\n\nRegardless of the approach, always conclude with:\n\nTherefore, the final answer is: $\\boxed{answer}$. I hope it is correct.\n\nWhere [answer] is just the final number or expression that solves the problem.'}, {'role': 'user', 'content': 'What is the smallest positive perfect cube that can be written as the sum of three consecutive integers?'}, {'role': 'assistant', 'content': "


Processed prompts: 100%|██████████| 1/1 [00:12<00:00, 12.77s/it, est. speed input: 23.27 toks/s, output: 160.44 toks/s]

  Generated: ## Step 3: Express the perfect cube in terms of x....
  Stop reason: eos_token
  Beam completed with stop reason: eos_token

Beams completed in this iteration: 4
Total completed beams so far: 4

-- Scoring beams --
Scoring 0 non-completed beams
All beams completed. Exiting search.

===== Search Complete =====
Total beams: 6 (Completed: 4, Active: 2)
Too many beams (6), keeping top 4

----- Final Results -----
Beam 1: last_score = 0.9922, completed = True, steps = 5
Full response: ## Step 1: To find the smallest positive perfect cube that can be written as the sum of three consecutive integers, let's start by defining the sum of three consecutive integers algebraically.

Let's denote the first of these consecutive integers as n, then the next two consecutive integers are n + 1 and n + 2. The sum of these three consecutive integers is given by n + (n + 1) + (n + 2).

## Step 2: Simplify the expression for the sum of three consecutive integers.

Combining like terms, the expr

## 6. Compare with Baseline (No Beam Search)

In [10]:
# Generate a baseline solution (no beam search)
system_prompt = "Solve the following math problem efficiently and clearly:\n\n- For simple problems (2 steps or fewer):\nProvide a concise solution with minimal explanation.\n\n- For complex problems (3 steps or more):\nUse this step-by-step format:\n\n## Step 1: [Concise description]\n[Brief explanation and calculations]\n\n## Step 2: [Concise description]\n[Brief explanation and calculations]\n\n...\n\nRegardless of the approach, always conclude with:\n\nTherefore, the final answer is: $\\boxed{answer}$. I hope it is correct.\n\nWhere [answer] is just the final number or expression that solves the problem."
baseline_text = create_chat_prompt(problem, system_prompt=system_prompt)

# Generate with vLLM
sampling_params = SamplingParams(
    temperature=0.8,
    max_tokens=1024,
    top_p=0.95,
)

outputs = llm.generate(baseline_text, sampling_params)
baseline_solution = outputs[0].outputs[0].text

# Show baseline result
print("Baseline Solution (No Beam Search):")
print("-" * 60)
print(baseline_solution)
print("-" * 60)

Gen conversation: 
 [{'role': 'system', 'content': 'Solve the following math problem efficiently and clearly:\n\n- For simple problems (2 steps or fewer):\nProvide a concise solution with minimal explanation.\n\n- For complex problems (3 steps or more):\nUse this step-by-step format:\n\n## Step 1: [Concise description]\n[Brief explanation and calculations]\n\n## Step 2: [Concise description]\n[Brief explanation and calculations]\n\n...\n\nRegardless of the approach, always conclude with:\n\nTherefore, the final answer is: $\\boxed{answer}$. I hope it is correct.\n\nWhere [answer] is just the final number or expression that solves the problem.'}, {'role': 'user', 'content': 'What is the smallest positive perfect cube that can be written as the sum of three consecutive integers?'}]


Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.24s/it, est. speed input: 55.22 toks/s, output: 152.39 toks/s]

Baseline Solution (No Beam Search):
------------------------------------------------------------
## Step 1: Understand the problem and identify the constraints
We need to find the smallest positive perfect cube that can be expressed as the sum of three consecutive integers.

## Step 2: Express a perfect cube as the sum of three consecutive integers
A perfect cube can be expressed as $n^3$ for some integer $n$. We need to find the smallest $n$ for which $n^3$ can be written as the sum of three consecutive integers.

## Step 3: Represent three consecutive integers
Let the three consecutive integers be $x-2$, $x$, and $x+1$.

## Step 4: Write the equation based on the given condition
We can write the equation: $(x-2) + x + (x+1) = n^3$

## Step 5: Simplify the equation
Combine like terms: $3x + 1 = n^3$

## Step 6: Test values of x to find the smallest n
Start with the smallest possible values for $x$ to find the smallest $n$ that satisfies the equation.

## Step 7: Test x = 1
For $x = 1$

In [11]:
# Show beam search results
print(f"Beam Search Results (Best {len(results[:3])} of {len(results)} beams):")
for i, beam in enumerate(results[:3]):
    print("=" * 80)
    print(f"Beam {i+1} (Score: {max(beam.scores) if beam.scores else 0:.4f})")
    print("-" * 60)
    print(beam.current_text)
    print("-" * 60)

Beam Search Results (Best 3 of 4 beams):
Beam 1 (Score: 0.9922)
------------------------------------------------------------
## Step 1: To find the smallest positive perfect cube that can be written as the sum of three consecutive integers, let's start by defining the sum of three consecutive integers algebraically.

Let's denote the first of these consecutive integers as n, then the next two consecutive integers are n + 1 and n + 2. The sum of these three consecutive integers is given by n + (n + 1) + (n + 2).

## Step 2: Simplify the expression for the sum of three consecutive integers.

Combining like terms, the expression simplifies to 3n + 3.

## Step 3: We need to find a positive integer n such that 3n + 3 is a perfect cube.

Let's start by trying values of n starting from 1, and for each value, check if 3n + 3 is a perfect cube.

## Step 4: Try n = 1.

3(1) + 3 = 6, which is not a perfect cube.

## Step 5: Try n = 2.

3(2) + 3 = 9, which is a perfect cube (3^2).

## Step 6: Sinc